# Winners Analysis - Council of Alphas

Pipeline: 3 strategies/family (12 total) - 1 champion/family (4 total) - 3 hybrids - Scientist loop - ranked survivors

Portfolio simulation: $100k initial capital, 0.5% risk per trade, ATR-based position sizing (TBM 2.0/1.0/24, 1h candles, MEXC 0.04% fee).

Dynamically loads whatever ranked results exist from `data/results/ranked/`.

In [1]:
import json
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path

INITIAL_CAPITAL = 100_000
RANKED_DIR = Path('../data/results/ranked')

# Load ranked summary to discover survivors dynamically
with open(RANKED_DIR / 'ranked_summary.json') as f:
    ranked_summary = json.load(f)

print(f'Survivors: {len(ranked_summary)}')
for s in ranked_summary:
    print(f'  #{s["rank"]}: {s["name"]} (fitness={s["fitness"]:.3f}, sharpe={s["sharpe"]:.4f}, trades={s["trades"]})')

# Load trade logs dynamically
state = pd.read_parquet('../data/state_matrix_1h.parquet')
long_outcomes = state['tbm_long_outcome'].astype(str).to_numpy(copy=False)
short_outcomes = state['tbm_short_outcome'].astype(str).to_numpy(copy=False)

def attach_tbm_outcome(df):
    out = df.copy()
    entries = out['entry_index'].to_numpy(dtype=np.int64)
    sides = out['side'].to_numpy(dtype=np.int8)
    outcomes = np.full(len(out), 'TIMEOUT', dtype=object)
    long_mask = sides == 1
    short_mask = sides == -1
    outcomes[long_mask] = long_outcomes[entries[long_mask]]
    outcomes[short_mask] = short_outcomes[entries[short_mask]]
    out['tbm_outcome'] = outcomes
    return out

palette = ['#00CC96', '#636EFA', '#EF553B', '#AB63FA', '#FFA15A']

strategies = {}
colors = {}
for i, s in enumerate(ranked_summary):
    rank = s['rank']
    name = s['name']
    label = f'{name.replace("_", " ").title()} (#{rank})'
    trade_log_path = RANKED_DIR / f'{rank}_{name}' / 'trade_log.csv'
    df = pd.read_csv(trade_log_path, parse_dates=['entry_ts', 'exit_ts'])
    strategies[label] = attach_tbm_outcome(df)
    colors[label] = palette[i % len(palette)]

print(f'\nLoaded {len(strategies)} strategies')

Survivors: 3
  #1: consensus_gate (fitness=0.502, sharpe=0.1252, trades=450)
  #2: regime_router (fitness=0.263, sharpe=0.0524, trades=2847)
  #3: weighted_combination (fitness=0.064, sharpe=0.0124, trades=4181)

Loaded 3 strategies


## 1. Head - First Trades

In [2]:
for name, df in strategies.items():
    print(f'=== {name} ({len(df):,} trades) ===')
    display(df.head(10))
    print()

=== Consensus Gate (#1) (450 trades) ===


,entry_ts,exit_ts,entry_index,exit_index,duration,side,net_trade_return,account_balance,session,trend_regime,vol_regime,tbm_outcome
0,2022-01-05 18:00:00+00:00,2022-01-05 19:00:00+00:00,114,115,1,-1,0.009802,100980.174597,NY,DOWNTREND,LOW_VOL,TP
1,2022-01-06 05:00:00+00:00,2022-01-06 17:00:00+00:00,125,137,12,-1,-0.005100,100465.203354,ASIA,DOWNTREND,HIGH_VOL,SL
2,2022-01-11 22:00:00+00:00,2022-01-12 07:00:00+00:00,262,271,9,1,-0.005145,99948.294936,OTHER,DOWNTREND,LOW_VOL,SL
3,2022-01-12 22:00:00+00:00,2022-01-13 01:00:00+00:00,286,289,3,1,-0.005144,99434.159014,OTHER,UPTREND,HIGH_VOL,SL
4,2022-01-13 06:00:00+00:00,2022-01-13 13:00:00+00:00,294,301,7,1,0.009864,100414.967512,ASIA,UPTREND,HIGH_VOL,TP
5,2022-01-31 03:00:00+00:00,2022-01-31 13:00:00+00:00,723,733,10,-1,-0.005132,99899.638264,ASIA,DOWNTREND,LOW_VOL,SL
6,2022-02-07 04:00:00+00:00,2022-02-07 07:00:00+00:00,892,895,3,1,-0.005135,99386.680983,ASIA,UPTREND,HIGH_VOL,SL
7,2022-02-07 13:00:00+00:00,2022-02-07 21:00:00+00:00,901,909,8,1,-0.005129,98876.953802,NY,UPTREND,HIGH_VOL,SL
8,2022-02-13 19:00:00+00:00,2022-02-13 20:00:00+00:00,1051,1052,1,-1,-0.005137,98369.030778,NY,DOWNTREND,LOW_VOL,SL
9,2022-02-13 21:00:00+00:00,2022-02-14 06:00:00+00:00,1053,1062,9,-1,-0.005148,97862.614601,OTHER,DOWNTREND,LOW_VOL,SL



=== Regime Router (#2) (2,847 trades) ===


,entry_ts,exit_ts,entry_index,exit_index,duration,side,net_trade_return,account_balance,session,trend_regime,vol_regime,tbm_outcome
0,2022-01-03 21:00:00+00:00,2022-01-03 22:00:00+00:00,69,70,1,-1,-0.005216,99478.350681,OTHER,DOWNTREND,HIGH_VOL,SL
1,2022-01-03 23:00:00+00:00,2022-01-04 04:00:00+00:00,71,76,5,-1,0.009789,100452.096912,OTHER,DOWNTREND,HIGH_VOL,TP
2,2022-01-04 21:00:00+00:00,2022-01-05 10:00:00+00:00,93,106,13,-1,-0.005177,99932.073870,OTHER,DOWNTREND,HIGH_VOL,SL
3,2022-01-05 17:00:00+00:00,2022-01-05 18:00:00+00:00,113,114,1,-1,-0.005193,99413.165348,NY,DOWNTREND,LOW_VOL,SL
4,2022-01-05 19:00:00+00:00,2022-01-05 21:00:00+00:00,115,117,2,-1,0.009827,100390.106921,NY,DOWNTREND,HIGH_VOL,TP
5,2022-01-05 22:00:00+00:00,2022-01-05 23:00:00+00:00,118,119,1,-1,-0.005122,99875.872122,OTHER,DOWNTREND,HIGH_VOL,SL
6,2022-01-06 00:00:00+00:00,2022-01-06 01:00:00+00:00,120,121,1,1,-0.005117,99364.842757,ASIA,DOWNTREND,HIGH_VOL,SL
7,2022-01-06 02:00:00+00:00,2022-01-06 03:00:00+00:00,122,123,1,1,-0.005111,98857.035050,ASIA,DOWNTREND,HIGH_VOL,SL
8,2022-01-06 04:00:00+00:00,2022-01-06 10:00:00+00:00,124,130,6,1,-0.005102,98352.678837,ASIA,DOWNTREND,HIGH_VOL,SL
9,2022-01-06 13:00:00+00:00,2022-01-06 18:00:00+00:00,133,138,5,-1,-0.005092,97851.869214,NY,DOWNTREND,HIGH_VOL,SL



=== Weighted Combination (#3) (4,181 trades) ===


,entry_ts,exit_ts,entry_index,exit_index,duration,side,net_trade_return,account_balance,session,trend_regime,vol_regime,tbm_outcome
0,2022-01-01 23:00:00+00:00,2022-01-02 04:00:00+00:00,23,28,5,-1,0.009782,100978.202181,OTHER,CONSOLIDATION,LOW_VOL,TP
1,2022-01-03 19:00:00+00:00,2022-01-03 20:00:00+00:00,67,68,1,-1,0.009760,101963.765111,NY,DOWNTREND,HIGH_VOL,TP
2,2022-01-03 21:00:00+00:00,2022-01-03 22:00:00+00:00,69,70,1,-1,-0.005216,101431.871824,OTHER,DOWNTREND,HIGH_VOL,SL
3,2022-01-03 23:00:00+00:00,2022-01-04 04:00:00+00:00,71,76,5,-1,0.009789,102424.740144,OTHER,DOWNTREND,HIGH_VOL,TP
4,2022-01-04 05:00:00+00:00,2022-01-04 07:00:00+00:00,77,79,2,-1,-0.005202,101891.968602,ASIA,DOWNTREND,HIGH_VOL,SL
5,2022-01-04 14:00:00+00:00,2022-01-04 16:00:00+00:00,86,88,2,1,-0.005178,101364.372076,NY,DOWNTREND,HIGH_VOL,SL
6,2022-01-04 18:00:00+00:00,2022-01-04 19:00:00+00:00,90,91,1,-1,-0.005170,100840.278757,NY,DOWNTREND,HIGH_VOL,SL
7,2022-01-04 20:00:00+00:00,2022-01-05 09:00:00+00:00,92,105,13,-1,-0.005173,100318.622985,NY,DOWNTREND,HIGH_VOL,SL
8,2022-01-05 13:00:00+00:00,2022-01-05 17:00:00+00:00,109,113,4,-1,0.009795,101301.269890,NY,DOWNTREND,LOW_VOL,TP
9,2022-01-05 18:00:00+00:00,2022-01-05 19:00:00+00:00,114,115,1,-1,0.009802,102294.199204,NY,DOWNTREND,LOW_VOL,TP


## 2. Info - DataFrame Structure

In [3]:
for name, df in strategies.items():
    print(f'=== {name} ===')
    print(f'Shape: {df.shape}')
    print()
    df.info()
    print('\n')

=== Consensus Gate (#1) ===
Shape: (450, 12)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   entry_ts          450 non-null    datetime64[ns, UTC]
 1   exit_ts           450 non-null    datetime64[ns, UTC]
 2   entry_index       450 non-null    int64              
 3   exit_index        450 non-null    int64              
 4   duration          450 non-null    int64              
 5   side              450 non-null    int64              
 6   net_trade_return  450 non-null    float64            
 7   account_balance   450 non-null    float64            
 8   session           450 non-null    object             
 9   trend_regime      450 non-null    object             
 10  vol_regime        450 non-null    object             
 11  tbm_outcome       450 non-null    object             
dtypes: datetime64[ns, 

## 3. Portfolio Performance

In [4]:
# net_trade_return is now the LEVERAGED portfolio return (not raw asset return)
# account_balance tracks compounding equity
for name, df in strategies.items():
    returns = df['net_trade_return']
    outcomes = df['tbm_outcome']
    final_equity = df['account_balance'].iloc[-1]
    total_return = (final_equity - INITIAL_CAPITAL) / INITIAL_CAPITAL
    
    # Compounded check (should match account_balance)
    comp_return = (1 + returns).prod() - 1
    
    # Win/loss based on TBM outcomes only (TP/SL). Timeouts excluded.
    tp_returns = returns[outcomes == 'TP']
    sl_returns = returns[outcomes == 'SL']
    timeout_count = int((outcomes == 'TIMEOUT').sum())
    wins = int(len(tp_returns))
    losses = int(len(sl_returns))
    scored = wins + losses
    wr = (wins / scored) if scored > 0 else np.nan
    avg_win = tp_returns.mean()
    avg_loss = sl_returns.mean()
    rr = avg_win / abs(avg_loss) if pd.notna(avg_loss) and avg_loss != 0 else np.nan
    
    # Drawdown
    peak = df['account_balance'].cummax()
    drawdown = (df['account_balance'] - peak) / peak
    max_dd = drawdown.min()
    
    print(f'=== {name} ===')
    print(f'  Total trades:       {len(df):,}')
    print(f'  Final equity:       ${final_equity:,.2f}')
    print(f'  Total return:       {total_return:+.2%}')
    print(f'  Max drawdown:       {max_dd:.2%}')
    print(f'  Win rate (TP/SL):   {wr:.1%}' if pd.notna(wr) else '  Win rate (TP/SL):   N/A')
    print(f'  TP hits / SL hits:  {wins:,} / {losses:,}')
    print(f'  Timeouts:           {timeout_count:,}')
    print(f'  Avg win (portfolio):{avg_win*100:>+10.4f}%')
    print(f'  Avg loss (portfolio):{avg_loss*100:>+9.4f}%')
    print(f'  Risk-reward:        {rr:.2f}')
    print()

=== Consensus Gate (#1) ===
  Total trades:       450
  Final equity:       $149,128.66
  Total return:       +49.13%
  Max drawdown:       -5.93%
  Win rate (TP/SL):   40.4%
  TP hits / SL hits:  178 / 263
  Timeouts:           9
  Avg win (portfolio):   +0.9846%
  Avg loss (portfolio):  -0.5158%
  Risk-reward:        1.91

=== Regime Router (#2) ===
  Total trades:       2,847
  Final equity:       $270,389.15
  Total return:       +170.39%
  Max drawdown:       -23.22%
  Win rate (TP/SL):   36.7%
  TP hits / SL hits:  1,021 / 1,759
  Timeouts:           67
  Avg win (portfolio):   +0.9832%
  Avg loss (portfolio):  -0.5164%
  Risk-reward:        1.90

=== Weighted Combination (#3) ===
  Total trades:       4,181
  Final equity:       $130,061.52
  Total return:       +30.06%
  Max drawdown:       -21.20%
  Win rate (TP/SL):   34.7%
  TP hits / SL hits:  1,415 / 2,665
  Timeouts:           101
  Avg win (portfolio):   +0.9834%
  Avg loss (portfolio):  -0.5162%
  Risk-reward:        1.

## 4. Sharpe Ratio Analysis

In [5]:
# Load state matrix to compute per-trade leverage (needed for zero-fee estimate)
state = pd.read_parquet('../data/state_matrix_1h.parquet')
close_arr = state['close'].values
atr_arr = state['ATR_24'].values

FEE = 0.00040  # MEXC taker fee
TBM_LOSS = 1.0
RISK_PCT = 0.005

for name, df in strategies.items():
    returns = df['net_trade_return']
    
    # -- Standard annualized Sharpe (daily resampled, sqrt(365) for crypto) --
    daily = df.set_index('entry_ts')['net_trade_return'].resample('1D').sum().fillna(0)
    ann_sharpe = (daily.mean() / daily.std()) * np.sqrt(365) if daily.std() > 0 else 0
    
    # Per-trade Sharpe (raw)
    mean_ret = returns.mean()
    std_ret = returns.std()
    per_trade_sharpe = mean_ret / std_ret if std_ret > 0 else 0
    
    # Zero-fee estimate: add back leverage * fee per trade
    entries = df['entry_index'].values
    sl_pct = TBM_LOSS * atr_arr[entries] / close_arr[entries]
    leverage = np.minimum(RISK_PCT / sl_pct, 20.0)
    gross_returns = returns + leverage * FEE
    
    gross_mean = gross_returns.mean()
    gross_std = gross_returns.std()
    gross_sharpe = gross_mean / gross_std if gross_std > 0 else 0
    
    print(f'=== {name} ===')
    print(f'  Annualized Sharpe:      {ann_sharpe:.4f}  (daily resampled, sqrt(365))')
    print(f'  Per-trade Sharpe:       {per_trade_sharpe:.4f}')
    print(f'  Zero-fee per-trade:     {gross_sharpe:.4f}')
    print(f'  Mean return per trade:  {mean_ret:.6f} ({mean_ret*100:.4f}%)')
    print(f'  Std dev of returns:     {std_ret:.6f} ({std_ret*100:.4f}%)')
    print(f'  Avg leverage:           {leverage.mean():.2f}x')
    print(f'  Skewness:               {returns.skew():.4f}')
    print(f'  Kurtosis:               {returns.kurtosis():.4f}')
    print()

=== Consensus Gate (#1) ===
  Annualized Sharpe:      1.2904  (daily resampled, sqrt(365))
  Per-trade Sharpe:       0.1251
  Zero-fee per-trade:     0.1465
  Mean return per trade:  0.000915 (0.0915%)
  Std dev of returns:     0.007317 (0.7317%)
  Avg leverage:           0.39x
  Skewness:               0.3888
  Kurtosis:               -1.8438

=== Regime Router (#2) ===
  Annualized Sharpe:      1.4000  (daily resampled, sqrt(365))
  Per-trade Sharpe:       0.0524
  Zero-fee per-trade:     0.0754
  Mean return per trade:  0.000375 (0.0375%)
  Std dev of returns:     0.007159 (0.7159%)
  Avg leverage:           0.41x
  Skewness:               0.5433
  Kurtosis:               -1.6839

=== Weighted Combination (#3) ===
  Annualized Sharpe:      0.3881  (daily resampled, sqrt(365))
  Per-trade Sharpe:       0.0124
  Zero-fee per-trade:     0.0354
  Mean return per trade:  0.000088 (0.0088%)
  Std dev of returns:     0.007075 (0.7075%)
  Avg leverage:           0.41x
  Skewness:           

## 5. Dollar Equity Curve ($100k start, 0.5% risk, MEXC 0.04%)

In [6]:
fig = make_subplots(
    rows=len(strategies), cols=1,
    subplot_titles=list(strategies.keys()),
    vertical_spacing=0.08,
)

for i, (name, df) in enumerate(strategies.items(), 1):
    fig.add_trace(
        go.Scatter(
            x=df['entry_ts'],
            y=df['account_balance'],
            mode='lines',
            name=name,
            line=dict(color=colors[name], width=1.5),
        ),
        row=i, col=1,
    )
    
    fig.add_hline(y=INITIAL_CAPITAL, line_dash='dash', line_color='gray', opacity=0.5, row=i, col=1)
    fig.update_yaxes(title_text='Equity ($)', tickformat='$,.0f', row=i, col=1)

fig.update_xaxes(title_text='Date', row=len(strategies), col=1)

fig.update_layout(
    height=300 * len(strategies),
    title_text='Dollar Equity Curve - $100k, 0.5% Risk Per Trade (TBM 2.0/1.0/24, 1h, MEXC 0.04%)',
    showlegend=True,
    template='plotly_white',
)

fig.show()

In [7]:
# Overlay comparison
fig2 = go.Figure()

for name, df in strategies.items():
    fig2.add_trace(
        go.Scatter(
            x=df['entry_ts'],
            y=df['account_balance'],
            mode='lines',
            name=name,
            line=dict(color=colors[name], width=1.5),
        )
    )

fig2.add_hline(y=INITIAL_CAPITAL, line_dash='dash', line_color='gray', opacity=0.5)

fig2.update_layout(
    height=500,
    title_text='Equity Overlay - All Three Hybrids ($100k Start)',
    yaxis_title='Equity ($)',
    yaxis_tickformat='$,.0f',
    xaxis_title='Date',
    template='plotly_white',
    legend=dict(x=0.01, y=0.99),
)

fig2.show()

## 6. Average Holding Time

In [8]:
# duration column is in bars (1h candles)
for name, df in strategies.items():
    holding_hours = df['duration']  # 1 bar = 1 hour on 1h data
    
    print(f'=== {name} ===')
    print(f'  Mean:   {holding_hours.mean():.2f} hours')
    print(f'  Median: {holding_hours.median():.2f} hours')
    print(f'  Min:    {holding_hours.min()} hours')
    print(f'  Max:    {holding_hours.max()} hours')
    print(f'  Hit time barrier (duration=24 bars / 24h): {(df["duration"] == 24).sum():,} ({(df["duration"] == 24).mean():.1%})')
    print()

=== Consensus Gate (#1) ===
  Mean:   6.41 hours
  Median: 5.00 hours
  Min:    1 hours
  Max:    24 hours
  Hit time barrier (duration=24 bars / 24h): 9 (2.0%)

=== Regime Router (#2) ===
  Mean:   5.95 hours
  Median: 4.00 hours
  Min:    1 hours
  Max:    24 hours
  Hit time barrier (duration=24 bars / 24h): 72 (2.5%)

=== Weighted Combination (#3) ===
  Mean:   5.87 hours
  Median: 4.00 hours
  Min:    1 hours
  Max:    24 hours
  Hit time barrier (duration=24 bars / 24h): 115 (2.8%)

